# IBM Introduction to Machine Learning
## Unsupervised Learning - Course Final Project

## FINDING THE BEST LOCATION FOR A NEW BUSINESS

First we download all we need to explore the data.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## Explore Dataset

Import all subway stations and its locations from Buenos Aires Government's web.

In [2]:
ba_subte = pd.read_csv("http://cdn.buenosaires.gob.ar/datosabiertos/datasets/subte-estaciones/estaciones-de-subte.csv")
ba_subte.head()

,long,lat,id,estacion,linea
0,-58.398928,-34.635750,1,CASEROS,H
1,-58.400970,-34.629376,2,INCLAN,H
2,-58.402323,-34.623092,3,HUMBERTO 1º,H
3,-58.404732,-34.615242,4,VENEZUELA,H
4,-58.406036,-34.608935,5,ONCE - 30 DE DICIEMBRE,H


Define Foursquare Credentials and Version (Hidden Cell).

In [3]:
# @hidden_cell

Create a map of Buenos Aires Subway Stations.

In [4]:
latitude = -34.6131516
longitude = -58.3772316

# create map of Buenos Aires using latitude and longitude values
ba_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, long, linea, estacion in zip(ba_subte['lat'], ba_subte['long'], ba_subte['linea'], ba_subte['estacion']):
    label = '{}, {}'.format(estacion, linea)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(ba_map)  
    
ba_map

Explore the first station in our dataframe and get latitude and longitude values.

In [5]:
ba_subte.loc[0, 'estacion']

'CASEROS'

In [6]:
station_latitude = ba_subte.loc[0, 'lat'] # station latitude value
station_longitude = ba_subte.loc[0, 'long'] # station longitude value

station_name = ba_subte.loc[0, 'estacion'] # station name

print('Latitude and longitude values of {} are {}, {}.'.format(station_name, 
                                                               station_latitude, 
                                                               station_longitude))

Latitude and longitude values of CASEROS are -34.635750180342995, -58.3989275853601.


Get the top 100 venues near CASEROS.

In [7]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    station_latitude, 
    station_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=HZZ1WGWJN01G4XC1DVCCB3H0FWATX3FQSZZINPAZBA5IIIS5&client_secret=HJHDSQIQK4LXZWDUCLKZHGIAIHOEBWRLDE022S5EGOCTQ1U4&v=20180605&ll=-34.635750180342995,-58.3989275853601&radius=500&limit=100'

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d5855390d2be7002c9594d3'},
 'response': {'headerLocation': 'Parque Patricios',
  'headerFullLocation': 'Parque Patricios, Buenos Aires',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': -34.63125017584299,
    'lng': -58.39346854148654},
   'sw': {'lat': -34.640250184843, 'lng': -58.404386629233656}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d9f403fa77d816e8516fe08',
       'name': 'Torres Blancas',
       'location': {'address': 'Caseros 2502',
        'crossStreet': 'Manuel García',
        'lat': -34.63550330087713,
        'lng': -58.39657553662826,
        'labeledLatLngs': [{'label': 'display',
          'lat': -34.63550330087713,
          'lng': -58.39657553662826

Get category type and nearby venues near CASEROS.

In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Torres Blancas,Pizza Place,-34.635503,-58.396576
1,El Soñado,Restaurant,-34.638783,-58.401507
2,Quality Padel,Athletics & Sports,-34.632734,-58.396514
3,Parque Florentino Ameghino,Park,-34.635466,-58.396535
4,El Codo de Oro,Pizza Place,-34.632106,-58.398977
5,El Pipi,Argentinian Restaurant,-34.637735,-58.397971
6,Carrefour Express,Grocery Store,-34.635670,-58.401830
7,Metrobus - Estación Jujuy,Bus Stop,-34.631788,-58.400855
8,E & E Caffé,Café,-34.634986,-58.393583


In [11]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


## Exploring Subway Stations

Create a function to repeat the same process to all stations in Buenos Aires.

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Station', 
                  'Station Latitude', 
                  'Station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Create a new dataframe and check the data.

In [13]:
ba_venues = getNearbyVenues(names=ba_subte['estacion'],
                                   latitudes=ba_subte['lat'],
                                   longitudes=ba_subte['long']
                                  )

CASEROS
INCLAN
HUMBERTO 1º
VENEZUELA
ONCE - 30 DE DICIEMBRE
9 DE JULIO
FACULTAD DE MEDICINA
TRIBUNALES - TEATRO COLÓN
AGÜERO
R.SCALABRINI ORTIZ
PLAZA ITALIA
PALERMO
RETIRO
LAVALLE
DIAGONAL NORTE
AV. DE MAYO
MORENO
INDEPENDENCIA
CONSTITUCION
FLORIDA
C. PELLEGRINI
URUGUAY
CALLAO
PASTEUR - AMIA
PUEYRREDON
CARLOS GARDEL
MEDRANO
ANGEL GALLARDO
MALABIA - OSVALDO PUGLIESE
DORREGO
PERU
PIEDRAS
LIMA
SAENZ PEÑA
CONGRESO
PASCO
ALBERTI
PLAZA DE MISERERE
LORIA
CASTRO BARROS
RIO DE JANEIRO
ACOYTE
PRIMERA JUNTA
BOLIVAR
BELGRANO
INDEPENDENCIA
SAN JOSE
ENTRE RIOS - RODOLFO WALSH
PICHINCHA
JUJUY
URQUIZA
JOSE MARIA MORENO
EMILIO MITRE
BULNES
PUEYRREDON
CALLAO
SAN MARTIN
SAN JUAN
MEDALLA MILAGROSA
AV. LA PLATA
BOEDO
MINISTRO CARRANZA
OLLEROS
JOSE HERNANDEZ
JURAMENTO
FEDERICO LACROZE
PLAZA DE LOS VIRREYES - EVA PERON
VARELA
CATEDRAL
PLAZA DE MAYO
CONGRESO DE TUCUMAN
LEANDRO N. ALEM
TRONADOR - VILLA ORTÚZAR
DE LOS INCAS -PQUE. CHAS
CARABOBO
PUAN
CORRIENTES
PARQUE PATRICIOS
HOSPITALES
ECHEVERRÍA
JUAN MANUEL 

In [14]:
print(ba_venues.shape)
ba_venues.head(10)

(3985, 7)


,Station,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,CASEROS,-34.635750,-58.398928,Torres Blancas,-34.635503,-58.396576,Pizza Place
1,CASEROS,-34.635750,-58.398928,El Soñado,-34.638783,-58.401507,Restaurant
2,CASEROS,-34.635750,-58.398928,Quality Padel,-34.632734,-58.396514,Athletics & Sports
3,CASEROS,-34.635750,-58.398928,Parque Florentino Ameghino,-34.635466,-58.396535,Park
4,CASEROS,-34.635750,-58.398928,El Codo de Oro,-34.632106,-58.398977,Pizza Place
5,CASEROS,-34.635750,-58.398928,El Pipi,-34.637735,-58.397971,Argentinian Restaurant
6,CASEROS,-34.635750,-58.398928,Carrefour Express,-34.635670,-58.401830,Grocery Store
7,CASEROS,-34.635750,-58.398928,Metrobus - Estación Jujuy,-34.631788,-58.400855,Bus Stop
8,CASEROS,-34.635750,-58.398928,E & E Caffé,-34.634986,-58.393583,Café
9,INCLAN,-34.629376,-58.400970,El Horno de Medio Oriente,-34.628523,-58.399974,Middle Eastern Restaurant


In [15]:
ba_venues.groupby('Station').count()

,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Station,,,,,,
9 DE JULIO,97,97,97,97,97,97
ACOYTE,38,38,38,38,38,38
AGÜERO,58,58,58,58,58,58
ALBERTI,18,18,18,18,18,18
ANGEL GALLARDO,45,45,45,45,45,45
AV. DE MAYO,75,75,75,75,75,75
AV. LA PLATA,28,28,28,28,28,28
BELGRANO,47,47,47,47,47,47
BOEDO,29,29,29,29,29,29


In [16]:
print('There are {} uniques categories.'.format(len(ba_venues['Venue Category'].unique())))

There are 221 uniques categories.


## Analyze each station

In [17]:
# one hot encoding
ba_onehot = pd.get_dummies(ba_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ba_onehot['Station'] = ba_venues['Station'] 

# move neighborhood column to the first column
fixed_columns = [ba_onehot.columns[-1]] + list(ba_onehot.columns[:-1])
ba_onehot = ba_onehot[fixed_columns]

ba_onehot.head()

,Station,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Border Crossing,Botanical Garden,Bowling Alley,Breakfast Spot,Brewery,Bridge,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Doctor's Office,Donut Shop,Drugstore,Duty-free Shop,Electronics Store,Empanada Restaurant,Event Space,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Record Shop,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Theater,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Video Game Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,CASEROS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,CASEROS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [18]:
ba_onehot.shape

(3985, 222)

Group rows by station and by taking the mean of the frequency of occurrence of each category.

In [19]:
ba_grouped = ba_onehot.groupby('Station').mean().reset_index()
ba_grouped

,Station,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Border Crossing,Botanical Garden,Bowling Alley,Breakfast Spot,Brewery,Bridge,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Doctor's Office,Donut Shop,Drugstore,Duty-free Shop,Electronics Store,Empanada Restaurant,Event Space,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Record Shop,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Theater,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Video Game Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,9 DE JULIO,0.000000,0.000000,0.000000,0.000000,0.000000,0.041237,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010309,0.010309,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041237,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020619,0.000000,0.000000,0.000000,0.000000,0.000000,0.144330,0.010309,0.010309,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.103093,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010309,0.000000,0.000000,0.000000,0.000000,0.010309,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.010309,0.000000,0.000000,0.010309,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.010309,0.000000,0.0

In [20]:
ba_grouped.shape

(86, 222)

Print top 5 most common venues for each station.

In [21]:
num_top_venues = 5

for hood in ba_grouped['Station']:
    print("----"+hood+"----")
    temp = ba_grouped[ba_grouped['Station'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----9 DE JULIO----
         venue  freq
0         Café  0.14
1        Hotel  0.12
2  Coffee Shop  0.10
3      Theater  0.08
4  Pizza Place  0.07


----ACOYTE----
            venue  freq
0  Ice Cream Shop  0.16
1            Café  0.16
2     Coffee Shop  0.11
3          Bakery  0.11
4    Burger Joint  0.05


----AGÜERO----
                    venue  freq
0                  Bakery  0.12
1             Pizza Place  0.07
2                    Café  0.07
3          Ice Cream Shop  0.07
4  Argentinian Restaurant  0.05


----ALBERTI----
                  venue  freq
0           Pizza Place  0.17
1                  Café  0.17
2                   Gym  0.11
3             BBQ Joint  0.11
4  Fast Food Restaurant  0.06


----ANGEL GALLARDO----
                    venue  freq
0          Ice Cream Shop  0.09
1                     Bar  0.07
2  Argentinian Restaurant  0.07
3                Pharmacy  0.04
4              Restaurant  0.04


----AV. DE MAYO----
                    venue  freq
0               

Sort venues in descending order.

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create new dataframe and display the top 10 venues for each station.

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Station']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
ba_venues_sorted = pd.DataFrame(columns=columns)
ba_venues_sorted['Station'] = ba_grouped['Station']

for ind in np.arange(ba_grouped.shape[0]):
    ba_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ba_grouped.iloc[ind, :], num_top_venues)

ba_venues_sorted.head()

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,9 DE JULIO,Café,Hotel,Coffee Shop,Theater,Pizza Place,Restaurant,Argentinian Restaurant,Bookstore,Mediterranean Restaurant,Gym / Fitness Center
1,ACOYTE,Café,Ice Cream Shop,Bakery,Coffee Shop,Burger Joint,Gym,Gastropub,Tea Room,Sporting Goods Shop,Market
2,AGÜERO,Bakery,Pizza Place,Ice Cream Shop,Café,Argentinian Restaurant,Pharmacy,Coffee Shop,Miscellaneous Shop,Burger Joint,Hotel
3,ALBERTI,Café,Pizza Place,BBQ Joint,Gym,Train Station,Restaurant,Supermarket,Brewery,Japanese Restaurant,Coffee Shop
4,ANGEL GALLARDO,Ice Cream Shop,Argentinian Restaurant,Bar,Coffee Shop,Bakery,Gym,Restaurant,Café,Pizza Place,Pharmacy


## Cluster Stations

Run k-means to cluster the neighborhood into 7 clusters.

In [24]:
# set number of clusters
kclusters = 7

ba_grouped_clustering = ba_grouped.drop('Station', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ba_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 3, 3, 3, 0, 3, 0, 6, 0], dtype=int32)

Create another dataframe that includes the cluster and top 10 venues for each station.

In [25]:
# add clustering labels
ba_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ba_merged = ba_subte

# merge ba_grouped with ba_data to add latitude/longitude for each neighborhood
ba_merged = ba_merged.join(ba_venues_sorted.set_index('Station'), on='estacion')

ba_merged.head()

,long,lat,id,estacion,linea,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,-58.398928,-34.635750,1,CASEROS,H,2,Pizza Place,Bus Stop,Café,Park,Argentinian Restaurant,Grocery Store,Restaurant,Athletics & Sports,Empanada Restaurant,Duty-free Shop
1,-58.400970,-34.629376,2,INCLAN,H,5,Soccer Field,Convenience Store,Pizza Place,Middle Eastern Restaurant,Yoga Studio,Drugstore,Food & Drink Shop,Flower Shop,Fish Market,Fast Food Restaurant
2,-58.402323,-34.623092,3,HUMBERTO 1º,H,2,Bus Stop,Pharmacy,Ice Cream Shop,Pizza Place,Theater,Bakery,Japanese Restaurant,Furniture / Home Store,Dessert Shop,Soccer Field
3,-58.404732,-34.615242,4,VENEZUELA,H,4,BBQ Joint,Convenience Store,Bakery,Yoga Studio,Pharmacy,Café,Gas Station,Concert Hall,Hardware Store,Doctor's Office
4,-58.406036,-34.608935,5,ONCE - 30 DE DICIEMBRE,H,4,Café,Electronics Store,Fast Food Restaurant,Pharmacy,BBQ Joint,Convenience Store,Peruvian Restaurant,Platform,Restaurant,Bus Stop


Visualize the clusters.

In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ba_merged['lat'], ba_merged['long'], ba_merged['estacion'], ba_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [32]:
ba_merged.loc[ba_merged['Cluster Labels'] == 0, ba_merged.columns[[3] + list(range(5, ba_merged.shape[1]))]]

,estacion,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,9 DE JULIO,0,Café,Hotel,Coffee Shop,Theater,Pizza Place,Restaurant,Argentinian Restaurant,Bookstore,Mediterranean Restaurant,Gym / Fitness Center
6,FACULTAD DE MEDICINA,0,Coffee Shop,Hotel,Italian Restaurant,Café,Community Center,Burger Joint,Sandwich Place,Bus Stop,Restaurant,Cheese Shop
7,TRIBUNALES - TEATRO COLÓN,0,Coffee Shop,Theater,Hotel,Café,Argentinian Restaurant,Pizza Place,Bakery,BBQ Joint,Shopping Plaza,Vegetarian / Vegan Restaurant
12,RETIRO,0,Hotel,Coffee Shop,Argentinian Restaurant,Italian Restaurant,Café,Deli / Bodega,Monument / Landmark,Sandwich Place,Restaurant,Plaza
13,LAVALLE,0,Coffee Shop,Café,Hotel,Pizza Place,Restaurant,Theater,Argentinian Restaurant,Italian Restaurant,Gym / Fitness Center,Bookstore
14,DIAGONAL NORTE,0,Coffee Shop,Café,Hotel,Theater,Pizza Place,Restaurant,Pharmacy,Burger Joint,Gym / Fitness Center,Ice Cream Shop
15,AV. DE MAYO,0,Hotel,Café,Spanish Restaurant,Argentinian Restaurant,Coffee Shop,Hostel,Burger Joint,Sandwich Place,Sports Club,Bar
16,MORENO,0,Spanish Restaurant,Hotel,Argentinian Restaurant,Café,Hostel,Camera Store,Sandwich Place,Burger Joint,Coffee Shop,Steakhouse
17,INDEPENDENCIA,0,Hotel,Hostel,Restaurant,Argentinian Restaurant,Spanish Restaurant,Coffee Shop,Café,Italian Restaurant,Burger Joint,Pub
19,FLORIDA,0,Coffee Shop,Café,Hotel,Argentinian Restaurant,Pizza Place,Restaurant,Salad Place,Theater,Deli / Bodega,Gym / Fitness Center


In [33]:
ba_merged.loc[ba_merged['Cluster Labels'] == 1, ba_merged.columns[[3] + list(range(5, ba_merged.shape[1]))]]

,estacion,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,MEDALLA MILAGROSA,1,Athletics & Sports,Korean Restaurant,Pizza Place,Ice Cream Shop,Border Crossing,Metro Station,Latin American Restaurant,Fabric Shop,Electronics Store,Empanada Restaurant
66,PLAZA DE LOS VIRREYES - EVA PERON,1,Soccer Field,Metro Station,Moving Target,Plaza,Big Box Store,Gym,Light Rail Station,Duty-free Shop,Flower Shop,Fish Market
67,VARELA,1,Metro Station,Pizza Place,Soccer Field,Supermarket,Light Rail Station,Athletics & Sports,Big Box Store,Fast Food Restaurant,Fish Market,Farmers Market


In [34]:
ba_merged.loc[ba_merged['Cluster Labels'] == 2, ba_merged.columns[[3] + list(range(5, ba_merged.shape[1]))]]

,estacion,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,CASEROS,2,Pizza Place,Bus Stop,Café,Park,Argentinian Restaurant,Grocery Store,Restaurant,Athletics & Sports,Empanada Restaurant,Duty-free Shop
2,HUMBERTO 1º,2,Bus Stop,Pharmacy,Ice Cream Shop,Pizza Place,Theater,Bakery,Japanese Restaurant,Furniture / Home Store,Dessert Shop,Soccer Field
18,CONSTITUCION,2,Bus Stop,Hotel,Pizza Place,Restaurant,Indie Movie Theater,Café,Intersection,Bus Station,Platform,Fabric Shop
48,PICHINCHA,2,Pizza Place,Argentinian Restaurant,Grocery Store,Diner,Japanese Restaurant,Butcher,Big Box Store,Furniture / Home Store,Bakery,Soccer Field
49,JUJUY,2,Ice Cream Shop,Bus Stop,Grocery Store,Pizza Place,Theater,Convenience Store,Japanese Restaurant,Breakfast Spot,Soccer Field,Furniture / Home Store
50,URQUIZA,2,Ice Cream Shop,Plaza,Pizza Place,Breakfast Spot,Coffee Shop,Grocery Store,Athletics & Sports,Theater,Gym / Fitness Center,Art Gallery


In [35]:
ba_merged.loc[ba_merged['Cluster Labels'] == 3, ba_merged.columns[[3] + list(range(5, ba_merged.shape[1]))]]

,estacion,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,AGÜERO,3,Bakery,Pizza Place,Ice Cream Shop,Café,Argentinian Restaurant,Pharmacy,Coffee Shop,Miscellaneous Shop,Burger Joint,Hotel
9,R.SCALABRINI ORTIZ,3,Pizza Place,Café,Coffee Shop,Ice Cream Shop,Bakery,Gym,Argentinian Restaurant,Deli / Bodega,Pharmacy,Plaza
11,PALERMO,3,Hotel,Ice Cream Shop,Bakery,Café,Argentinian Restaurant,Pizza Place,Gym / Fitness Center,Nightclub,Bike Rental / Bike Share,Clothing Store
23,PASTEUR - AMIA,3,Café,Japanese Restaurant,Gym,Coffee Shop,Arts & Crafts Store,Gift Shop,Korean Restaurant,Middle Eastern Restaurant,Sandwich Place,Salad Place
24,PUEYRREDON,3,Café,Pizza Place,Bakery,Pharmacy,Gym,Argentinian Restaurant,Coffee Shop,Ice Cream Shop,Sandwich Place,Hotel
26,MEDRANO,3,Restaurant,Pizza Place,Café,Bar,Argentinian Restaurant,Vegetarian / Vegan Restaurant,Gym,Bike Rental / Bike Share,Rock Club,Dessert Shop
27,ANGEL GALLARDO,3,Ice Cream Shop,Argentinian Restaurant,Bar,Coffee Shop,Bakery,Gym,Restaurant,Café,Pizza Place,Pharmacy
28,MALABIA - OSVALDO PUGLIESE,3,Pizza Place,Coffee Shop,Argentinian Restaurant,Ice Cream Shop,Café,Italian Restaurant,Bus Stop,Gym,Shoe Store,Burger Joint
36,ALBERTI,3,Café,Pizza Place,BBQ Joint,Gym,Train Station,Restaurant,Supermarket,Brewery,Japanese Restaurant,Coffee Shop
40,RIO DE JANEIRO,3,Pizza Place,Bakery,Pharmacy,Ice Cream Shop,Dessert Shop,Coffee Shop,Sandwich Place,Gym,Restaurant,Pub


In [36]:
ba_merged.loc[ba_merged['Cluster Labels'] == 4, ba_merged.columns[[3] + list(range(5, ba_merged.shape[1]))]]

,estacion,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,VENEZUELA,4,BBQ Joint,Convenience Store,Bakery,Yoga Studio,Pharmacy,Café,Gas Station,Concert Hall,Hardware Store,Doctor's Office
4,ONCE - 30 DE DICIEMBRE,4,Café,Electronics Store,Fast Food Restaurant,Pharmacy,BBQ Joint,Convenience Store,Peruvian Restaurant,Platform,Restaurant,Bus Stop
25,CARLOS GARDEL,4,Bar,Ice Cream Shop,Café,Argentinian Restaurant,Theater,Big Box Store,Italian Restaurant,Gastropub,Bed & Breakfast,Frozen Yogurt Shop
29,DORREGO,4,Café,Bakery,Soccer Field,Athletics & Sports,Soccer Stadium,Sporting Goods Shop,Beer Garden,Sandwich Place,Latin American Restaurant,Park
37,PLAZA DE MISERERE,4,Café,Hotel,Fast Food Restaurant,BBQ Joint,Gym,Gas Station,Electronics Store,Convenience Store,Peruvian Restaurant,Restaurant
46,SAN JOSE,4,Hostel,Soccer Field,Theme Restaurant,Fast Food Restaurant,Bus Stop,Café,Gay Bar,Pool,Grocery Store,Plaza


In [37]:
ba_merged.loc[ba_merged['Cluster Labels'] == 5, ba_merged.columns[[3] + list(range(5, ba_merged.shape[1]))]]

,estacion,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,INCLAN,5,Soccer Field,Convenience Store,Pizza Place,Middle Eastern Restaurant,Yoga Studio,Drugstore,Food & Drink Shop,Flower Shop,Fish Market,Fast Food Restaurant


In [38]:
ba_merged.loc[ba_merged['Cluster Labels'] == 6, ba_merged.columns[[3] + list(range(5, ba_merged.shape[1]))]]

,estacion,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,PLAZA ITALIA,6,Argentinian Restaurant,Hotel,Pizza Place,Bistro,Pub,Breakfast Spot,Botanical Garden,French Restaurant,Fast Food Restaurant,Nail Salon
38,LORIA,6,Argentinian Restaurant,Ice Cream Shop,Italian Restaurant,Bar,Pizza Place,Hotel,Cheese Shop,Soccer Field,Sandwich Place,Restaurant
39,CASTRO BARROS,6,Argentinian Restaurant,Bar,Ice Cream Shop,Pizza Place,Hotel,BBQ Joint,Restaurant,Café,Cheese Shop,Spanish Restaurant
47,ENTRE RIOS - RODOLFO WALSH,6,Argentinian Restaurant,Bakery,Pizza Place,Metro Station,Theme Restaurant,Gay Bar,Diner,Supermarket,Plaza,Food Court
51,JOSE MARIA MORENO,6,Pizza Place,Brewery,Argentinian Restaurant,Ice Cream Shop,Bar,Beer Bar,Soccer Field,Burger Joint,Auto Dealership,Plaza
52,EMILIO MITRE,6,Argentinian Restaurant,Ice Cream Shop,Park,Coffee Shop,Fish Market,Brewery,Breakfast Spot,Restaurant,Soccer Field,Electronics Store
60,BOEDO,6,Argentinian Restaurant,Pizza Place,Breakfast Spot,Café,Cosmetics Shop,Pharmacy,Brewery,Cafeteria,Soccer Field,Plaza
65,FEDERICO LACROZE,6,Pizza Place,Argentinian Restaurant,Office,Bar,Bakery,Spanish Restaurant,Coffee Shop,Ice Cream Shop,Pharmacy,Venezuelan Restaurant
78,HOSPITALES,6,Pharmacy,Ice Cream Shop,Argentinian Restaurant,Seafood Restaurant,Tennis Court,Donut Shop,Flower Shop,Fish Market,Fast Food Restaurant,Farmers Market


### Area near stations from Cluster 0 (or RED) clearly seems to be the best positioned to develop the new business.